In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.

import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import notMNIST_gen

In [2]:
save = notMNIST_gen.gen_composite()
train_dataset = save['train_dataset']
train_labels = save['train_labels']
valid_dataset = save['valid_dataset']
valid_labels = save['valid_labels']
test_dataset = save['test_dataset']
test_labels = save['test_labels']
del save  # hint to help gc free up memory
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Starting
Download Complete
Extract Complete
Saving Complete
Data Dictionaries Built
Training set (200000, 64, 64) (200000,)
Validation set (10000, 64, 64) (10000,)
Test set (10000, 64, 64) (10000,)


In [3]:
labels = train_labels
new_labels = labels.view('S1').reshape((labels.size, -1))
print(new_labels[0])

[b'A' b'B' b'E' b'H' b' ']


In [4]:
image_size = 64
num_digits = 5
char_labels = [b'A',b'B',b'C',b'D',b'E',b'F',b'G',b'H',b'I',b'J',b' ']
num_labels = len(char_labels)
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = labels.view('S1').reshape((labels.size, -1))
  new_labels = np.ndarray(labels.shape, dtype=np.float32)
  for i in range(new_labels.shape[0]):
    for j in range(new_labels.shape[1]):
        if labels[i,j] in char_labels:
            new_labels[i,j] = char_labels.index(labels[i,j])
  labels = (np.arange(num_labels) == new_labels[:,:,None])
  labels = labels.astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 64, 64, 1) (200000, 5, 11)
Validation set (10000, 64, 64, 1) (10000, 5, 11)
Test set (10000, 64, 64, 1) (10000, 5, 11)


In [5]:
print(train_labels[0])

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]


In [6]:
batch_size = 32
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
  with tf.name_scope('input_variables'):
      tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels), 
          name='train_dataset_in')
      tf_train_labels = tf.placeholder(
          tf.float32, shape=(batch_size, num_digits, num_labels), name='train_labels_in')
      tf_valid_dataset = tf.constant(valid_dataset, name='valid_dataset_in')
      tf_test_dataset = tf.constant(test_dataset, name='valid_labels_in')

  def cnn_var_dict():
      var_dict = {}
      with tf.name_scope('convolutional_net_variables'):
          var_dict['layer_1_weights'] = tf.Variable(tf.truncated_normal(
              [patch_size, patch_size, num_channels, depth], stddev=0.1), 
                                                   name='cnn_layer_1_weights')
          var_dict['layer_1_biases'] = tf.Variable(tf.zeros([depth]), 
                                                  name='cnn_layer_1__biases')
          var_dict['layer_2_weights'] = tf.Variable(tf.truncated_normal(
              [patch_size, patch_size, depth, depth], stddev=0.1), 
                                                   name='cnn_layer_2_weights')
          var_dict['layer_2_biases'] = tf.Variable(tf.constant(1.0, shape=[depth]), 
                                                  name='cnn_layer_2_biases')
          for key in var_dict:
            tf.summary.histogram(key + '_summary', var_dict[key])
      return var_dict

  def variable_dict(digit):
      var_dict = {}
      with tf.name_scope('Digit_' + digit + '_variables'):
          var_dict['layer_1_weights'] = tf.Variable(tf.truncated_normal(
              [image_size // 4 * image_size // 4 * depth, num_hidden*4], stddev=0.1), 
                                                   name='layer_1_'+ digit + '_weights')
          var_dict['layer_1_biases'] = tf.Variable(tf.constant(1.0, shape=[num_hidden*4]), 
                                                  name='layer_1_'+ digit + '_biases')
          var_dict['layer_2_weights'] = tf.Variable(tf.truncated_normal(
              [num_hidden*4, num_hidden*3], stddev=0.1), 
                                                   name='layer_2_'+ digit + '_weights')
          var_dict['layer_2_biases'] = tf.Variable(tf.constant(1.0, shape=[num_hidden*3]), 
                                                  name='layer_2_'+ digit + '_biases')
          var_dict['layer_3_weights'] = tf.Variable(tf.truncated_normal(
              [num_hidden*3, num_hidden*2], stddev=0.1), 
                                                   name='layer_3_'+ digit + '_weights')
          var_dict['layer_3_biases'] = tf.Variable(tf.constant(1.0, shape=[num_hidden*2]), 
                                                  name='layer_3_'+ digit + '_biases')
          var_dict['layer_4_weights'] = tf.Variable(tf.truncated_normal(
              [num_hidden*2, num_labels], stddev=0.1), name='layer_4_'+ digit + '_weights')
          var_dict['layer_4_biases'] = tf.Variable(tf.constant(1.0, shape=[num_labels]), 
                                                  name='layer_4_'+ digit + '_biases')
          for key in var_dict:
            tf.summary.histogram(key + '_summary', var_dict[key])
      return var_dict

  cnn_vars = cnn_var_dict()
  deep_vars = {}
  for i in range(num_digits):
        deep_vars[str(i)] = variable_dict(str(i))
                      
  # Model.
  def cnn_model(data, var_set):
    with tf.name_scope('convolutional_net_operations'):
        conv = tf.nn.conv2d(data, var_set['layer_1_weights'], [1, 2, 2, 1], padding='SAME', 
                            name='Convolution_1')
        conv = tf.nn.max_pool(conv, [1, 4, 4, 1], [1, 1, 1, 1], padding='SAME', name='Max_Pool_1')
        hidden = tf.nn.relu(conv + var_set['layer_1_biases'], name='CNN_Relu_1')
        conv = tf.nn.conv2d(hidden, var_set['layer_2_weights'], [1, 2, 2, 1], padding='SAME', 
                            name='Convolution_2')
        conv = tf.nn.max_pool(conv, [1, 4, 4, 1], [1, 1, 1, 1], padding='SAME', name='Max_Pool_2')
        hidden = tf.nn.relu(conv + var_set['layer_2_biases'], name='CNN_Relu_2')
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]], name='Collapse_to_2d')
    return reshape

  def digit_model(data, digit):
      with tf.name_scope('Digit_' + digit + '_operations'):
        hidden_1 = tf.nn.relu(tf.matmul(data, deep_vars[digit]['layer_1_weights']) + 
                              deep_vars[digit]['layer_1_biases'], 
                              name='Digit_' + digit + '_Deep_Relu_1')
        hidden_2 = tf.nn.relu(tf.matmul(hidden_1, deep_vars[digit]['layer_2_weights']) + 
                              deep_vars[digit]['layer_2_biases'], 
                              name='Digit_' + digit + '_Deep_Relu_2')
        hidden_3 = tf.nn.relu(tf.matmul(hidden_2, deep_vars[digit]['layer_3_weights']) + 
                              deep_vars[digit]['layer_3_biases'], 
                              name='Digit_' + digit + '_Deep_Relu_3')
        result = tf.matmul(hidden_3, deep_vars[digit]['layer_4_weights']) + \
                            deep_vars[digit]['layer_4_biases']
      return result

  def model(data):
      with tf.name_scope('model'):
          cnn = cnn_model(data, cnn_vars)
          logit_stack = list()
          for i in range(num_digits):
              logit_stack.append(digit_model(cnn, str(i)))
          logits = tf.stack(logit_stack, axis=1)
      return logits

  # Training computation.
  with tf.name_scope('training_computation'):
      logits = model(tf_train_dataset)
#      logit_shape = logits.get_shape().as_list()
#      label_shape = tf_train_labels.get_shape().as_list()
#      logit_reshape = tf.reshape(logits,[logit_shape[0]*logit_shape[1], logit_shape[2]])
#      label_reshape = tf.reshape(tf_train_labels,[label_shape[0]*label_shape[1], label_shape[2]])
      loss = tf.reduce_max(
#        tf.nn.softmax_cross_entropy_with_logits(logit_reshape, label_reshape), 
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels), 
          name='loss_function')    

  with tf.name_scope('optimizer_computation'):
      global_step = tf.Variable(0, name='global_step')  # count the number of steps taken.
      learning_rate = tf.train.exponential_decay(0.001, global_step, 10000, 0.95, name='learning_rate')
      optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step, 
                                                                 name='optimizer')
  
  # Predictions for the training, validation, and test data.
  with tf.name_scope('predictions'):
      train_prediction = tf.nn.softmax(logits, name='training_prediction')
      valid_prediction = tf.nn.softmax(model(tf_valid_dataset), name='validation_prediction')
      test_prediction = tf.nn.softmax(model(tf_test_dataset), name='testing_prediction')
        
#  with tf.name_scope('accuracy'):
#    correct_pred = np.argmax(predictions, 1) == np.argmax(labels, 1)
#    correct_addr = np.all(correct_pred, 1)
#    accuracy_op = tf.reduce_mean(tf.cast(correct_addr, "float"))
#    tf.scalar_summary('accuracy', accuracy_op)

In [ ]:
def accuracy(predictions, labels):
  correct_digits = np.argmax(predictions, 2) == np.argmax(labels, 2)
  correct_addrs = np.all(correct_digits, 1)
#  print(np.argmax(predictions, 2)[0], np.argmax(labels, 2)[0])
#  print(correct_digits[0, :], correct_addrs[0])
  return (100.0 * np.sum(correct_addrs) / predictions.shape[0])

In [ ]:
num_steps = 500001

with tf.Session(graph=graph) as session:
  writer = tf.summary.FileWriter('logs/nn_logs', graph=graph)
  merged = tf.summary.merge_all()
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :, :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, summary, l, predictions, learn = session.run(
      [optimizer, merged, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    writer.add_summary(summary, step)
    if (step % 10000 == 0):
      print('Minibatch loss at step %d: %f; Learing Rate: %f' % (step, l, learn))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))